In [1]:
%matplotlib qt
import hyperspy.api as hs
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import glob

In [3]:
# import data (Preisach acquisition curves for all Lunca samples)

filenames1 = sorted(glob.glob('*TM_P_mom_irm_norm.txt'))

In [4]:
filenames1

['L1_TM_P_mom_irm_norm.txt',
 'L2_TM_P_mom_irm_norm.txt',
 'L3_TM_P_mom_irm_norm.txt',
 'L4_TM_P_mom_irm_norm.txt',
 'L5_TM_P_mom_irm_norm.txt',
 'L6_TM_P_mom_irm_norm.txt',
 'S0_TM_P_mom_irm_norm.txt',
 'S1_TM_P_mom_irm_norm.txt',
 'S2_TM_P_mom_irm_norm.txt',
 'S3_TM_P_mom_irm_norm.txt',
 'S4_TM_P_mom_irm_norm.txt',
 'S5_TM_P_mom_irm_norm.txt',
 'S6_TM_P_mom_irm_norm.txt']

In [5]:
# gather all data together in the same array

dataP=[]
for file_path in filenames1:
    dataP.append(np.genfromtxt(file_path))
dataP=np.array(dataP)
dataP.shape

(13, 50, 50)

In [6]:
# plot data no 3 for check

plt.plot(dataP[3])

In [7]:
# normalize to 1 

dataPnn=np.zeros((13,50,50))
for i in range(len(dataP)):
    for j in range(50):
        dataPnn[i,:,j]=dataP[i,:,j]/np.max(dataP[i,:,j])
    #print(np.max(dataP[:,i,:]))#/np.diff(np.log10(h))
print(np.max(dataPnn[3],axis=0))
plt.plot(dataPnn[3])

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1.]


In [8]:
# load data as 2D signal in hyperspy

s=hs.signals.Signal2D(dataPnn)
s

<Signal2D, title: , dimensions: (13|50, 50)>

In [9]:
# SVD decomposition

s.decomposition()

Decomposition info:
  normalize_poissonian_noise=False
  algorithm=SVD
  output_dimension=None
  centre=None


In [10]:
# determine the number of end-members by calculating the cumulative variance given by SVD

a=s.get_explained_variance_ratio()
var=a.data
#var
varnorm=np.cumsum(var)
varnorm
pc_idx=np.linspace(1,10,10)
plt.plot(pc_idx,varnorm[0:10]*100,'o')
plt.xticks(np.arange(1,11,step=1))
plt.xlabel('Principal Component index')
plt.ylabel('Cumulative Variance (%)')

Text(0, 0.5, 'Cumulative Variance (%)')

In [11]:
# import Fastica module

from sklearn.decomposition import FastICA

In [103]:
# apply Fastica to the first two PCs provided by SVD

s.blind_source_separation(number_of_components=2,algorithm=FastICA(algorithm='parallel',random_state=1,fun='exp'),diff_order=3)
#s.blind_source_separation(number_of_components=2,algorithm='orthomax',diff_order=2)


[########################################] | 100% Completed |  0.1s


[########################################] | 100% Completed |  0.1s
Blind source separation info:
  number_of_components=2
  algorithm=FastICA(fun='exp', random_state=1)
  diff_order=3
  reverse_component_criterion=factors
  whiten_method=PCA
scikit-learn estimator:
FastICA(fun='exp', random_state=1)


In [104]:
#s.plot_bss_results()

In [105]:
# get EM1 and plot it

factors=s.get_bss_factors()
em1=factors.inav[0].data
plt.plot(em1)

In [96]:
# get EM2 and plot it (skip it for diff_order=3)

em2=factors.inav[1].data
plt.plot(em2)

In [97]:
#em3=factors.inav[2].data
#plt.plot(em3)

In [106]:
# Calculate Preisach map and plot it (see Church et al., 2016)

def poza_Preisach_mean(data1,fields1):    
    app_fields=np.mean(fields1,axis=0)
    #cond_fields=np.genfromtxt('field', delimiter=',', skip_header=93, skip_footer=1,usecols=(0))
    cond_fields=fields1[0,:]
    cond_fields[0]=app_fields[0]
    data2=data1
    data3=np.zeros((len(data1),len(data1)))
    lastpnt=len(data1)-1
    ref=data2[lastpnt,0]
    #print(ref)
    #print(cond_fields)
    #print(app_fields)
    #for i in range(len(data1)):
        #for j in range(1,len(data1)):
            #data3[i,0]=((ref/data2[0,0])*((lastpnt-i)/lastpnt))+((ref/data2[lastpnt,0])*(i/lastpnt))
            #data3[i,j]=(ref/data2[lastpnt,j-1])*((lastpnt-i)/lastpnt)+(ref/data2[lastpnt,j])*(i/lastpnt)
    #data3=np.array(data3)
    #if method == 'Preisach':
        #data3=data3*data2
    #elif method == 'FORC':
        #data3=data2
    #else:
            #raise ValueError("Method must be 'Preisach' or 'FORC'.")       
    data3=data1
    #print(data3[:,0]) 
    deltaM=np.zeros((len(data1)-1, len(data1)-1))
    for i in range(len(data1)-1):
        for j in range(len(data1)-1):
            deltaM[i,j]=((data3[i,j]+data3[i+1,j+1])-(data3[i,j+1]+data3[i+1,j]))*(
                (app_fields[i+1]+app_fields[i])*(cond_fields[j+1]+cond_fields[j]))/(
               (app_fields[i+1]-app_fields[i])*(cond_fields[j+1]-cond_fields[j]))/8
    deltaM=np.array(deltaM)
    deltaM=deltaM/np.max(deltaM)
    #return(deltaM,app_fields)
    #%matplotlib notebook
    from matplotlib import cm
    from matplotlib.colors import ListedColormap
    from matplotlib.colors import LinearSegmentedColormap
    import matplotlib.colors as colors
    def custom_div_cmap(numcolors=128, name='custom_div_cmap',
                    col1='blue', col2='white', col3 = 'green', col4='yellow',col5='red',col6='purple'):
        
        cmap = LinearSegmentedColormap.from_list(
                    name=name, 
                    colors=[col1, col2, col3, col4 ,col5,col6], 
                    N=numcolors
                )
    
        return cmap 
    (cmap)=custom_div_cmap(numcolors=128, name='custom_div_cmap',
                    col1='blue', col2='white', col3 = 'green', col4='yellow',col5='red',col6='purple')
    colors_undersea = cmap(np.linspace(0, 0.199, 128))
    colors_land = cmap(np.linspace(0.2, 1, 128))
    divnorm = colors.TwoSlopeNorm(vmin=np.min(deltaM), vcenter=0, vmax=np.max(deltaM))
    all_colors = np.vstack((colors_undersea, colors_land))
    terrain_map = colors.LinearSegmentedColormap.from_list('terrain_map',
        all_colors)
    fig, ax = plt.subplots()
    cs = ax.pcolor(app_fields[1:]*1000, cond_fields[1:]*1000,np.transpose(deltaM[0:,0:]),cmap=terrain_map,norm=divnorm)
    plt.yscale('log')
    plt.xscale('log')
    #plt.ylim(max(app_fields[1:]*1000),min(app_fields[1:])*1000)
    plt.ylim(max(app_fields[1:]*1000),min(app_fields[1:])*1000)
    cbar = fig.colorbar(cs)
    #fig2, plt.plot(np.sum(np.transpose(deltaM)))
    return np.transpose(deltaM)

In [107]:
# load field values for Preisach maps

l1=np.genfromtxt('COS-L1_NLP50steps_0.1-1000mT_3sec.csv',delimiter=',',skip_header=241,usecols=(3,4))
l1f=np.reshape(l1[:,0],(50,50))
l1p=np.reshape(l1[:,1],(50,50))

In [108]:
#plot EM1 Preisach. Inspect the EM1 Preisach map structure and if you are satisfied with it 
#(correspond to physically plausible Preisach pattern) you can keep it. If no then perform again Fastica 
# using a higher derivative order (see the cells bellow). In our case Em1 has no physically plausible aspect (negative region across diagonal)

em1p=poza_Preisach_mean(em1,l1f)

In [109]:
# plot EM2 Preisach. Inspect the EM2 Preisach map structure and if you are satisfied with it 
#(correspond to physically plausible Preisach pattern) you can keep it. If no then perform again Fastica 
# using a higher derivative order. In our case the second derivative provide a physically plausible EM2. (this reproduce Fig. 4a) 

em2p=poza_Preisach_mean(em2,l1f)

In [102]:
# perform fastica using diff order = 2 by setting diff_order = 2 in s.blind_source_separation above and assign the resulted 
# em2 to final EM2 (this reproduce Fig. 5a) (skip it for diff_order=3)

em2_diff2=em2
em2p=poza_Preisach_mean(em2_diff2,l1f)

In [110]:
# perform fastica using diff order = 3 by setting diff_order = 3 in s.blind_source_separation above and assign the resulted 
# em1 to final EM1 (this reproduce Fig. 5b)

em1_diff3=em1
em1p=poza_Preisach_mean(em1_diff3,l1f)

In [111]:
# calculate and plot the remanence gradient from Preisach maps. (this reproduce Fig. 5c)

plt.plot(l1f[1][1:]*1000,np.sum(em2p,axis=0)/np.max(np.sum(em2p)))
plt.plot(l1f[1][1:]*1000,np.sum(em1p,axis=0)/np.max(np.sum(em1p)))
plt.xscale('log')
plt.text(3,0.06,'~22mT',fontsize=15)
plt.text(80,0.06,'~65mT',fontsize=15)

Text(80, 0.06, '~65mT')

In [112]:
# normalize EM1

em1_diff3n=np.zeros((50,50))
for i in range(len(em1_diff3)):
    em1_diff3n[i,:]=em1_diff3[i,:]/np.max(em1_diff3[i,:])
    #print(np.max(dataP[:,i,:]))#/np.diff(np.log10(h))

#em1_diff3n = em1_diff3/np.max(em1_diff3)
print(np.max(em1_diff3n,axis=1))
plt.plot(em1_diff3n)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1.]


In [113]:
# normalize EM2

em2_diff2n=np.zeros((50,50))
for i in range(len(em2_diff2)):
    em2_diff2n[i,:]=em2_diff2[i,:]/np.max(em2_diff2[i,:])
    #print(np.max(dataP[:,i,:]))#/np.diff(np.log10(h))
    
#em2_diff2n = em2_diff2/np.max(em2_diff2)
print(np.max(em2_diff2n,axis=1))
plt.plot(em2_diff2n)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1.]


In [114]:
# normalized initial data

data_init=dataPnn.reshape((13,2500))
data_init.shape

(13, 2500)

In [115]:
# vectorize EM2

em2_diff2n_n=em2_diff2n.flatten()
em2_diff2n_n.shape

(2500,)

In [116]:
# vectorize EM1

em1_diff3n_n=em1_diff3n.flatten()
em1_diff3n_n.shape

(2500,)

In [117]:
# stack them forming thus the ems matrix

ems=np.vstack((em2_diff2n_n,em1_diff3n_n))
ems.shape

(2, 2500)

In [118]:
# calculate the pseudo inverse of the ems matrix

ems_pinv=np.linalg.pinv(ems)

In [119]:
# calculate the relative contributions by matrix multiplication between the initial data and pseudo inverse of the ems matrix

ab=np.matmul(data_init,ems_pinv)
ab.shape

(13, 2)

In [120]:
# stack plot of the relative contributiona. (this reproduce Fig. 5d)

ab1ln=ab[:,0]/(ab[:,0]+ab[:,1])
ab2ln=ab[:,1]/(ab[:,0]+ab[:,1])

#ab1ln=ab[:,0]
#ab2ln=ab[:,1]
ab1n=np.array([ab1ln[6],ab1ln[0],ab1ln[7],ab1ln[1],ab1ln[8],ab1ln[2],ab1ln[9],ab1ln[3],ab1ln[10],ab1ln[4],ab1ln[11],ab1ln[5],ab1ln[12]])
ab2n=np.array([ab2ln[6],ab2ln[0],ab2ln[7],ab2ln[1],ab2ln[8],ab2ln[2],ab2ln[9],ab2ln[3],ab2ln[10],ab2ln[4],ab2ln[11],ab2ln[5],ab2ln[12]])
#print(em1ln)
#print(em2ln)

#em1ln+em2ln
#xx=np.linspace(1,13,13)

depth=np.genfromtxt('Lunca_depth_Preisach.txt')
#depth=pcomb[:,0]
plt.legend(loc='upper left')

plt.stackplot(depth,ab1n,ab2n)
plt.axhline(y=0.5,color='white')

In [59]:
# plot EM1

em1pn=poza_Preisach_mean(em1_diff3n,l1f)

In [60]:
#plot EM2

em2pn=poza_Preisach_mean(em2_diff2n,l1f)

In [133]:
np.savetxt('TM_EM1_Preisach_fastica.txt',ab1n,delimiter='\t')
np.savetxt('TM_EM2_Preisach_fastica.txt',ab2n,delimiter='\t')